In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('data/data_domains.csv') # from sql code cpp_domains_final and 
# bigquery code cpp_domains_statistics
govcms_data = pd.read_csv('data/govcms_data.csv')

In [3]:
# lets make it all one table
govcms_data = govcms_data.rename(columns={"domainname": "domain"})
tables = [data, govcms_data]
data_domains = pd.concat(tables)
data_domains = data_domains.set_index('domain')

In [48]:
data_domains = data_domains[data_domains.index != 'govcms']
data_domains['bounce_rate'] = data_domains['total_bounces']/data_domains['total_visitors']

In [49]:
data_domains.columns

Index(['total_visitors', 'unique_visitors', 'total_new_users',
       'percentage_new_visitors', 'total_pageviews', 'pages_per_session',
       'avg_time_on_page', 'total_bounces', 'traffic_medium_referral',
       'traffic_medium_organic', 'traffic_medium_direct', 'traffic_medium_cpm',
       'traffic_medium_email', 'traffic_medium_other',
       'traffic_medium_all_gov_referral', 'traffic_medium_fed_gov_referral',
       'traffic_medium_other_gov_referral', 'bounce_rate'],
      dtype='object')

In [62]:
def reject_outliers(data):
    if np.isnan(np.quantile(data,0.25)):
        Q1 = data.quantile(0.25)
        Q3 = data.quantile(0.75)
    else:
        Q1 = np.quantile(data,0.25)
        Q3 = np.quantile(data,0.75)
    IQR = Q3 - Q1
    return np.array([e for i,e in enumerate(data) if ((data > (Q1 - 1.5 * IQR)) & (data < (Q3 + 1.5 * IQR)))[i]])

In [66]:
def bucket_maker(data,column_name):
    bucketed_data = pd.cut(data,15)
    sorted_data = bucketed_data.value_counts()
    return_data = pd.DataFrame(sorted_data, columns= ['number in bucket'])
    return_data['metric'] = column_name
    return_data = return_data.reset_index()
    return return_data

In [81]:
metrics_to_use = ['percentage_new_visitors', 'pages_per_session', 'bounce_rate', 'avg_time_on_page']
all_data = pd.DataFrame(columns = ['index','number in bucket', 'metric'])
for current_metric in metrics_to_use:
    current_data = data_domains[current_metric]
    current_data = reject_outliers(current_data)
    current_data = bucket_maker(current_data,current_metric)
    tables = [all_data, current_data]
    all_data = pd.concat(tables)

In [83]:
all_data = all_data.rename(columns={'index':'bucket'})

In [85]:
all_data.to_csv('output_data.csv')